Este notebook nos permite encontrar palabras similares y con ello, generar nuestro normalizador para un corpus.

# Descarga e importación de bibliotecas

Descargamos las bibliotecas necesarias para el uso del notebook

In [ ]:
!pip install fuzzywuzzy
!pip install python-Levenshtein

In [ ]:
from fuzzywuzzy import fuzz, process # https://pypi.org/project/fuzzywuzzy/
from collections import Counter, defaultdict
import pickle
import string 

/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Procesamiento de datos


In [ ]:
# obtenemos los corpus
with open("/content/drive/MyDrive/Aux/limpio (1).txt","r") as file:
  corpus = file.read()

In [ ]:
corpus = corpus.replace("\n","")

In [ ]:
# sacamos nuestro vocabulario
lista = corpus.split(" ")
lista = list(filter(lambda x: len(x)>0, lista))
vocabulario = list(Counter(lista).keys())

In [ ]:
# Estadistica del vocabulario a trabajar
print("Longitud del vocabulario:",len(vocabulario))

In [ ]:
# Obtenemos un diccionario de longitudes para que sea mucho más rápida 
# y exacta la comparación con otras palabras
diccionario_longitudes_palabras = defaultdict(list)
for palabra in vocabulario:
  diccionario_longitudes_palabras[len(palabra)].append(palabra)

# Comparación de palabras en el corpus

In [ ]:
# Funciones para serializar
def guardar(nombre,datos):
  """
  Guarda los datos de un objeto de python

  Parámetros
  nombre: Nombre del archivo
  datos: Objeto a guardar
  """
  with open(nombre+'.pickle', 'wb') as f:
    pickle.dump(datos, f)

def cargar(nombre):
  """
  Carga los datos de un objeto de python

  Parámetros
  nombre: Nombre del archivo

  return objeto cargado
  """
  with open(nombre+'.pickle', 'rb') as f:
    entry = pickle.load(f)
  
  return entry

In [ ]:
def comparaciones(vocabulario, diccionario_longitudes_palabras, holgura, porcentaje, limite):
  """
  Realiza las comparaciones entre palabras

  Parámetros
  vocabulario: Lista de palabras diferentes en el corpus
  diccionario_longitudes_palabras: Diccionario longitud:[palabras con esa 
                                                        longitud]
  holgura: Con cuantos caracteres más y menos se comparará una longitud
  porcentaje: Porcentaje de similitud para determinar que las palabras son 
    iguales
  limite: Limite de palabras similares
  
  return conjuntos de palabras similares y palabras observadas
  """

  palabras_vistas = dict()

  conjuntos_similares = list()
  # Para cada palabra del vocabulario
  for palabra in vocabulario:
    if palabra in palabras_vistas.keys():
      continue
    # lista de similares a la palabra objetivo
    lista_similares = list()

    # buscamos solo en las palabras que tienen una longitud similar (dependiendo
    # de la holgura) a la palabra objetivo
    for longitud in range(len(palabra)-holgura, len(palabra)+holgura+1):
      # Si la longitud se encuentra entre las llaves del diccionario de 
      # longitudes
      if longitud in diccionario_longitudes_palabras.keys():
        # Sacamos la lista de palabras de esa longitud
        posibilidades = diccionario_longitudes_palabras[longitud]
        # similares sacamos las palabras similares y filtramos por el 
        # parámetro
        similares = process.extract(palabra, posibilidades, limit=limite)
        lista_similares.extend([x for x in similares if x[1]>porcentaje and x[0]!=palabra])

    # agregamos las similitudes
    if len(lista_similares)>0:
      conjuntos_similares.append((palabra,lista_similares))
    # filtramos palabras ya vistas
    for palabra in [(palabra,0)]+lista_similares:
      palabras_vistas[palabra[0]] = palabra[0]

    guardar("/content/drive/MyDrive/resultados_s.txt",conjuntos_similares)
    guardar("/content/drive/MyDrive/resultados_v.txt",palabras_vistas)
  
  return conjuntos_similares, palabras_vistas

# Pruebas

In [ ]:
conjuntos, palabras = comparaciones(vocabulario, diccionario_longitudes_palabras, 2, 85, 5)

In [ ]:
print(conjuntos[1])

In [ ]:
conjuntos

In [ ]:
a = "iztacihuatl"

holgura = 2
porcentaje = 85

lista_similares = list()
for longitud in range(len(a)-holgura, len(a)+holgura+1):
  if longitud in diccionario_longitudes_palabras.keys():
    choices = diccionario_longitudes_palabras[longitud]
    similares = process.extract(a, choices, limit=5)
    lista_similares.extend([x for x in similares if x[1]>porcentaje])